# 1. Installation of the Dependencies

## 1.1 Creating a Virtual Environment

To isolate the project dependencies and ensure that they are properly installed and configured, it is a best practice to create a virtual environment.

### What is a Virtual Environment?

A virtual environment is a self-contained directory that contains a specific version of Python and any packages or modules needed for a particular project. By using a virtual environment, you can isolate the dependencies of a project from the global Python environment on your machine, which makes it easier to manage and avoid version conflicts.

### Creating a Virtual Environment

You can create a virtual environment using either `conda` 

To create a new virtual environment called `tensorflow_env` using `conda`, run the following cell:


In [ ]:
!conda create --name tensorflow_env

### Activating and Deactivating a Virtual Environment

To activate the virtual environment, run the following cell:

In [ ]:
!conda activate tensorflow_env

To deactivate the virtual environment, run the following cell:

In [ ]:
!deactivate

## 1.2 Installing Required Packages

Once you have activated the virtual environment, install the required packages running the following cell:


In [30]:
#install dependencies in the new environment:
!conda install -c jmcmurray os
!conda install -c anaconda pathlib
!conda install -c conda-forge matplotlib
!conda install -c anaconda numpy
!conda install -c anaconda seaborn
!conda install -c conda-forge tensorflow
!conda install -c anaconda ipython
!conda install -c anaconda urllib3
!conda install -c anaconda zipfile36
!conda install -c anaconda scipy
!conda install -c conda-forge librosa
!conda install -c conda-forge pydub

Solving environment: done

# All requested packages already installed.



## 1.3 Importing the dependencies

In [1]:
import tensorflow as tf

2023-02-20 18:20:41.196639: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-20 18:20:41.276681: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-20 18:20:41.276711: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-20 18:20:41.713157: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [52]:
import os
import pathlib
import urllib.request
import zipfile

from scipy.io.wavfile import write

import matplotlib.pyplot as plt
import numpy as np
# import librosa
import subprocess
from pydub import AudioSegment
import seaborn as sns
import tensorflow as tf
import tensorflow_io as tfio

from tensorflow.keras import layers
from tensorflow.keras import models
from IPython import display

# Set the seed value for experiment reproducibility.
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)



#  2. Importing the MDVR-KCL Dataset

## 2.1 Project Overview
In this project, we will be working with the **Mobile Device Voice Recordings at King's College London (MDVR-KCL) dataset**, which includes voice recordings from both early and advanced Parkinson's disease patients as well as from healthy controls.

Our goal is to use machine learning to classify the recordings and differentiate between healthy controls and Parkinson's disease patients, as well as between early and advanced stages of the disease.

## 2.2 Dataset

To work with this dataset, we will be downloading a zipfile from the Zenodo data repository that contains the voice recordings. We will then extract the files and use them to train and test our machine learning models.

### Download
To **download** the dataset into your computer:

In [ ]:
#!TEST

# specify the URL to download the zip file
url = "https://zenodo.org/record/2867216/files/26_29_09_2017_KCL.zip?download=1"

# Define the subdirectory to extract the contents of the zip file
subdir = "data"

# create a subdirectory named "data" 
if not os.path.exists("data"):
    os.makedirs("data")
    print(f"Created subdir data")
    
# Download the zip file from the URL
filename, headers = urllib.request.urlretrieve(url)
print(f"Downloaded file")

# Extract the contents of the zip file to the subdirectory
with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall(subdir)
print(f"Extracted files to data")

# Remove the downloaded zip file
os.remove(filename)

print(f"Dataset downloaded and extracted to subdir data. Zip file removed")

### Lable the dataset

Lables used to distinguish between: 
- Healthy Control Subjects **(HC)**
- Subjects with Parkinson's disease **(PD)** 

In [9]:
labels = ['HC','PD']
print('Labels:', labels)

Labels: ['HC', 'PD']


### Extract audio files and shuffle

In [39]:
#Extract audio files
data_dir = pathlib.Path("data/26-29_09_2017_KCL")
filenames = tf.io.gfile.glob(str(data_dir / '*' / '*' / '*'))

# create a list of label indices for each file
label_indices = [labels.index(pathlib.Path(filename).parent.name) for filename in filenames]
# print(label_indices)

#Shuffle audio files to avoid bias
filenames_random = tf.random.shuffle(filenames)
num_samples = len(filenames_random)

# Create a tf.data.Dataset from the shuffled filenames
dataset = tf.data.Dataset.from_tensor_slices(filenames_random)


print('Number of total examples:', num_samples)
print('Number of examples in HC:',
      sum([len(tf.io.gfile.listdir(str(data_dir / subdir / labels[0]))) for subdir in ['ReadText', 'SpontaneousDialogue']]))
print('Number of examples in PD:',
      sum([len(tf.io.gfile.listdir(str(data_dir / subdir / labels[1]))) for subdir in ['ReadText', 'SpontaneousDialogue']]))

print('Example file tensor:', filenames_random[0])

Number of total examples: 73
Number of examples in HC: 42
Number of examples in PD: 31
Example file tensor: tf.Tensor(b'data/26-29_09_2017_KCL/SpontaneousDialogue/HC/ID09_hc_0_0_0.wav', shape=(), dtype=string)


## 3. Data Augmentation


Because we have a dataset of only 73 audio files, we are going to use various data augmentation techniques:
- Adding Gaussian background noise
- Changing the pitch of the recordings

### Convert the audio to 16bit

!!Cambiar. Primero convertir a 16 bits y despues crear el dataset. Conservar las carpetas??

In [78]:
# Navigate to the data directory
data_dir = 'data/26-29_09_2017_KCL/'

# Set the output directory for the 16-bit audio files
output_dir = 'data/audio_16bit'

# Create the output directory if it does not exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
#get the path of the output dir
output_dir = os.path.abspath(os.path.join(os.getcwd(),output_dir))

# Save the current working directory
original_dir = os.getcwd()

#change dir to get the files
os.chdir(data_dir)

# Convert all WAV files to 16-bit
for subdir in ['ReadText/HC', 'ReadText/PD', 'SpontaneousDialogue/HC', 'SpontaneousDialogue/PD']:
    for file in os.listdir(subdir):
        if file.endswith('.wav'):
            input_file = os.path.join(subdir, file)
            print('INPUT FILE: ',input_file)
            output_file = os.path.join(output_dir, f"{os.path.splitext(file)[0]}.16.wav")
            print('OUTPUT FILE: ', output_file)
            subprocess.call(['ffmpeg', '-i', input_file, '-ac', '1', '-ar', '16000', '-sample_fmt', 's16', output_file])
           
# Change back to the original directory
os.chdir(original_dir)

INPUT FILE:  ReadText/HC/ID22_hc_0_0_0.wav
OUTPUT FILE:  /home/julia/Documents/UNI/3/2CUATRI/Biometria/Practica/data/audio_16bit/ID22_hc_0_0_0.16.wav
INPUT FILE:  ReadText/HC/ID28_hc_0_0_0.wav
OUTPUT FILE:  /home/julia/Documents/UNI/3/2CUATRI/Biometria/Practica/data/audio_16bit/ID28_hc_0_0_0.16.wav


ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

INPUT FILE:  ReadText/HC/ID09_hc_0_0_0.wav
OUTPUT FILE:  /home/julia/Documents/UNI/3/2CUATRI/Biometria/Practica/data/audio_16bit/ID09_hc_0_0_0.16.wav
INPUT FILE:  ReadText/HC/ID25_hc_0_0_0.wav
OUTPUT FILE:  /home/julia/Documents/UNI/3/2CUATRI/Biometria/Practica/data/audio_16bit/ID25_hc_0_0_0.16.wav


ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

INPUT FILE:  ReadText/HC/ID36_hc_0_0_0.wav
OUTPUT FILE:  /home/julia/Documents/UNI/3/2CUATRI/Biometria/Practica/data/audio_16bit/ID36_hc_0_0_0.16.wav
INPUT FILE:  ReadText/HC/ID11_hc_0_0_0.wav
OUTPUT FILE:  /home/julia/Documents/UNI/3/2CUATRI/Biometria/Practica/data/audio_16bit/ID11_hc_0_0_0.16.wav


...

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

INPUT FILE:  SpontaneousDialogue/PD/ID34_pd_2_0_0.wav
OUTPUT FILE:  /home/julia/Documents/UNI/3/2CUATRI/Biometria/Practica/data/audio_16bit/ID34_pd_2_0_0.16.wav
INPUT FILE:  SpontaneousDialogue/PD/ID24_pd_2_0_0.wav
OUTPUT FILE:  /home/julia/Documents/UNI/3/2CUATRI/Biometria/Practica/data/audio_16bit/ID24_pd_2_0_0.16.wav
INPUT FILE:  SpontaneousDialogue/PD/ID13_pd_3_2_2.wav
OUTPUT FILE:  /home/julia/Documents/UNI/3/2CUATRI/Biometria/Practica/data/audio_16bit/ID13_pd_3_2_2.16.wav
INPUT FILE:  SpontaneousDialogue/PD/ID04_pd_2_0_1.wav
OUTPUT FILE:  /home/julia/Documents/UNI/3/2CUATRI/Biometria/Practica/data/audio_16bit/ID04_pd_2_0_1.16.wav
INPUT FILE:  SpontaneousDialogue/PD/ID33_pd_3_2_2.wav
OUTPUT FILE:  /home/julia/Documents/UNI/3/2CUATRI/Biometria/Practica/data/audio_16bit/ID33_pd_3_2_2.16.wav
INPUT FILE:  SpontaneousDialogue/PD/ID20_pd_3_0_1.wav
OUTPUT FILE:  /home/julia/Documents/UNI/3/2CUATRI/Biometria/Practica/data/audio_16bit/ID20_pd_3_0_1.16.wav


ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

INPUT FILE:  SpontaneousDialogue/PD/ID27_pd_4_1_1.wav
OUTPUT FILE:  /home/julia/Documents/UNI/3/2CUATRI/Biometria/Practica/data/audio_16bit/ID27_pd_4_1_1.16.wav


ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

In [34]:
# def decode_wav_24bit(file_path):
#     # Read the audio file using tf.io.read_file
#     audio_binary = tf.io.read_file(file_path)
    
#     # Convert the 24-bit data to 16-bit data
#     audio_data = tf.io.decode_raw(audio_binary, tf.int24)
#     audio_data = tf.reshape(audio_data, [-1, 2])[:, ::-1]
#     audio_data = tf.cast(audio_data, tf.int16)
    
#     # Decode the WAV file using tf.audio.decode_wav
#     audio, sample_rate = tf.audio.decode_wav(tf.reshape(audio_data, [-1]))
#     audio = tf.cast(audio, dtype=tf.float32)
#     audio = tf.divide(audio, 32768.0)  # scale the audio to the range [-1, 1]
    
#     return audio, sample_rate


In [32]:
def adjust_pitch(audio, sample_rate, pitch_factor):
    # Convert the audio tensor to a numpy array
    audio = np.array(audio)

    # Convert the numpy array to an AudioSegment
    audio_segment = AudioSegment(
        data=audio.tobytes(),
        sample_width=audio.dtype.itemsize,
        frame_rate=sample_rate,
        channels=1
    )

    # Adjust the pitch using the pydub library
    adjusted_audio_segment = audio_segment._spawn(
        audio_segment.raw_data,
        overrides={
            "frame_rate": int(sample_rate * pitch_factor)
        }
    )
    adjusted_audio_segment = adjusted_audio_segment.set_frame_rate(sample_rate)

    # Convert the adjusted AudioSegment back to a numpy array
    adjusted_audio = np.frombuffer(adjusted_audio_segment.raw_data, np.int16)

    # Convert the numpy array back to a tensor
    adjusted_audio = tf.convert_to_tensor(adjusted_audio)

    # Normalize the audio tensor
    adjusted_audio = tf.cast(adjusted_audio, dtype=tf.float32)
    adjusted_audio = tf.divide(adjusted_audio, 32768.0)  # scale the audio to the range [-1, 1]

    return adjusted_audio

In [79]:

# Define a function to load the audio files and apply data augmentation
def load_audio_file(file_path, pitch_factor=0.0, noise_factor=0.0):
   
    # Load the audio file using tf.audio.decode_wav
    audio, sample_rate = tf.audio.decode_wav(tf.io.read_file(file_path))

     # Add background noise to the audio file
    if noise_factor > 0.0:
        noise = tf.random.normal(tf.shape(audio), stddev=noise_factor)
        audio = tf.cast(audio, dtype=tf.float32)
        audio = audio + noise


    # Change the pitch of the audio file
    if pitch_factor != 0.0:
        audio = change_pitch(audio, sample_rate, pitch_factor)

    # Return the audio data and label (if available)
    return audio,file_path

pitch_factor = 0.2
noise_factor = 0.1
# Use map to apply the load_audio_file function to the dataset
dataset = dataset.map(load_audio_file)

# Iterate over the dataset and print the file path for each element
for audio, file_path in dataset:
    print('audio:', audio, 'file_path:', file_path)

Tensor("args_0:0", shape=(), dtype=string)


2023-02-20 22:00:45.150160: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at decode_wav_op.cc:52 : INVALID_ARGUMENT: Can only read 16-bit WAV files, but received 24
2023-02-20 22:00:45.150586: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at decode_wav_op.cc:52 : INVALID_ARGUMENT: Can only read 16-bit WAV files, but received 24
2023-02-20 22:00:45.155789: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at decode_wav_op.cc:52 : INVALID_ARGUMENT: Can only read 16-bit WAV files, but received 24
2023-02-20 22:00:45.161476: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at decode_wav_op.cc:52 : INVALID_ARGUMENT: Can only read 16-bit WAV files, but received 24
2023-02-20 22:00:45.164372: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at decode_wav_op.cc:52 : INVALID_ARGUMENT: Can only read 16-bit WAV files, but received 24
2023-02-20 22:00:45.169494: W tensorflow/core/framework/op_kernel.cc:1780] 

InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Can only read 16-bit WAV files, but received 24
	 [[{{node DecodeWav}}]] [Op:IteratorGetNext]

### Divide files into training, validation and tests sets.

In [16]:
#! EL DATASET ES MUY PEQUE COMO PARA HACER ESTO. SOLUCIÓN APARTE
#Ratio of 80:10:10 respectively
train_files = filenames[:58]
val_files = filenames[58: 58 + 7]
test_files = filenames[-7:]

print('Training set size', len(train_files))
print('Validation set size', len(val_files))
print('Test set size', len(test_files))

Training set size 58
Validation set size 7
Test set size 7


Leer los archivos de audio y sus etiquetas.
En esta sección, preprocesará el conjunto de datos, creando tensores decodificados para las formas de onda y las etiquetas correspondientes. Tenga en cuenta que:

Cada archivo WAV contiene datos de series temporales con un número determinado de muestras por segundo.
Cada muestra representa la amplitud de la señal de audio en ese momento específico.
En un sistema de 16 bits , como los archivos WAV en el conjunto de datos mini Speech Commands, los valores de amplitud oscilan entre -32 768 y 32 767.
La frecuencia de muestreo para este conjunto de datos es de 16 kHz.
La forma del tensor devuelto por tf.audio.decode_wav es [samples, channels] , donde channels son 1 para mono o 2 para estéreo. El conjunto de datos de mini comandos de voz solo contiene grabaciones mono.



In [ ]:
test_file_path = 'data/26-29_09_2017_KCL/ReadText/PD/ID02_pd_2_0_0.wav'
test_file = tf.io.read_file(test_file_path)

test_audio, _ = tf.audio.decode_wav(contents=test_file)
test_audio.shape